## Check which NVIDIA-GPU you got

In [1]:
!nvidia-smi

Sun Dec  6 08:02:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!sudo apt-get install tree -qq > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


## Prerequisites

Let's start by installing some required libraries by the YOLOv5 project:

In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3189 (delta 11), reused 16 (delta 7), pack-reused 3165
Receiving objects: 100% (3189/3189), 6.46 MiB | 33.58 MiB/s, done.
Resolving deltas: 100% (2123/2123), done.


In [4]:
%cd yolov5
!pip install -r requirements.txt 

/content/yolov5
     |████████████████████████████████| 276kB 9.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=3405e67e6eada897e9790d4a714f7afcf88aa03cb5d6cbb32a3e853f6f1394dc
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


We'll also install [Apex by NVIDIA](https://nvidia.github.io/apex/) to speed up the training of our model (this step is optional):

In [5]:
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

Cloning into 'apex'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 7865 (delta 7), reused 10 (delta 0), pack-reused 7835
Receiving objects: 100% (7865/7865), 13.99 MiB | 26.38 MiB/s, done.
Resolving deltas: 100% (5356/5356), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-vcu_sq2u
Created temporary directory: /tmp/pip-req-tracker-2jcp1mdi
Created requirements tracker '/tmp/pip-req-tracker-2jcp1mdi'
Created temporary directory: /tmp/pip-install-fh7pkg2a
Processing /content/yolov5/apex
  Created temporary directory: /tmp/pip-req-build-3fdakway
  Added file:///content/yolov5/apex to build tracker '/tmp/pip-req-tracker-2jcp1mdi'
    Run

## Connect with drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Let's import all required libraries:

In [7]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

np.random.seed(42)

## Fine-tuning YOLO v5

The YOLO abbreviation stands for You Only Look Once. YOLO models are one stage object detectors. 

YOLO models are very light and fast. They are [not the most accurate object detections around](https://paperswithcode.com/sota/object-detection-on-coco), though. Ultimately, those models are the choice of many (if not all) practitioners interested in [real-time object detection (FPS >30)](https://paperswithcode.com/sota/real-time-object-detection-on-coco).

### Contreversy

Joseph Redmon introduced YOLO v1 in the 2016 paper [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/pdf/1506.02640.pdf). The implementation uses the [Darknet Neural Networks library](https://pjreddie.com/darknet/).

He also co-authored the YOLO v2 paper in 2017  [YOLO9000: Better, Faster, Stronger](https://arxiv.org/pdf/1612.08242.pdf). A significant improvement over the first iteration with much better localization of objects.

The final iteration, from the original author, was published in the 2018 paper [YOLOv3: An Incremental Improvement](https://arxiv.org/pdf/1804.02767.pdf).

Then things got a bit wacky. Alexey Bochkovskiy published [YOLOv4: Optimal Speed and Accuracy of Object Detection](https://arxiv.org/abs/2004.10934) on April 23, 2020. The project has [an open-source repository on GitHub](https://github.com/AlexeyAB/darknet).

YOLO v5 got open-sourced on [May 30, 2020](https://github.com/ultralytics/yolov5/commit/1e84a23f38fad9e52b59101e9f1246d93066ed1e) by [Glenn Jocher](https://github.com/glenn-jocher) from ultralytics. There is no published paper, but [the complete project is on GitHub](https://github.com/ultralytics/yolov5).


### YOLO v5 project setup

Let's start by cloning the GitHub repo and checking out a specific commit (to ensure reproducibility):

We need two configuration files. One for the dataset and one for the model we're going to use. Let's download them:

In [8]:
!gdown --id 1DVPEmt_RaFVfPZeNNW7IGHPZJSHaGBCL -O data/veh.yaml
!gdown --id 1RgXlGVRyOMkRCPo0qE-dqhUo-YlhUtkB -O models/yolov5l.yaml
!gdown --id 1WqKv3GkR4ImdAMcwKFtwnH8vXlYy6te1 -O models/yolov5s.yaml
#!gdown --id 1_D_izJB-ZJBowHLUMvG8-2LrklBmibs4 -O models/yolov5m.yaml
!gdown --id 1Q1kLhCgD_SvmHw2Gs5l3md8zz-xdPZ_C -O models/yolov5x.yaml

Downloading...
From: https://drive.google.com/uc?id=1DVPEmt_RaFVfPZeNNW7IGHPZJSHaGBCL
To: /content/yolov5/data/veh.yaml
100% 570/570 [00:00<00:00, 465kB/s]
Downloading...
From: https://drive.google.com/uc?id=1RgXlGVRyOMkRCPo0qE-dqhUo-YlhUtkB
To: /content/yolov5/models/yolov5l.yaml
100% 1.45k/1.45k [00:00<00:00, 1.26MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WqKv3GkR4ImdAMcwKFtwnH8vXlYy6te1
To: /content/yolov5/models/yolov5s.yaml
100% 1.46k/1.46k [00:00<00:00, 1.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Q1kLhCgD_SvmHw2Gs5l3md8zz-xdPZ_C
To: /content/yolov5/models/yolov5x.yaml
100% 1.58k/1.58k [00:00<00:00, 2.79MB/s]


### Training

To train a model on a custom dataset, we'll call the `train.py` script. We'll pass a couple of parameters:

- img 640 - resize the images to 640x640 pixels
- batch 4 - 4 images per batch
- epochs 30 - train for 30 epochs
- data ./data/veh.yaml - path to dataset config
- cfg ./models/yolov5s.yaml - model config
- weights yolov5s.pt - use pre-trained weights from the YOLOv5s model
- name yolov5s_clothing - name of our model
- cache - cache dataset images for faster training


function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)


In [ ]:

function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)

In [10]:
%%capture

!python train.py --img 640 --batch 16 --epochs 460 --data ./data/veh.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_veh --cache

## Evaluation

The project includes a great utility function `plot_results()` that allows you to evaluate your model performance on the last training run:

In [ ]:
from utils.general import plot_results         
plot_results();                                            

## Making predictions

In [15]:
#!python new_detect2.py --weights weights/best.pt --img 640 --conf 0.50 --source ./data/images/
!python new_detect2.py --weights runs/train/yolov5s_veh/weights/last.pt --img 1024 --conf 0.20 --source ./data/images/

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.2, device='', img_size=1024, iou_thres=0.45, save_conf=False, save_dir='inference/output', save_txt=False, source='./data/images/', update=False, view_img=False, weights=['runs/train/yolov5s_veh/weights/last.pt'])
Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)

Fusing layers... 
Model Summary: 232 layers, 7300458 parameters, 0 gradients
image 1/450 /content/yolov5/data/images/Asraf_100_jpg.rf.472d5bd950587612520051eebd827328.jpg: 1024x1024 11 rickshaws, 1 three wheelers (CNG)s, 1 minivans, 1 bicycles, 2 minibuss, Done. (0.020s)
image 2/450 /content/yolov5/data/images/Asraf_101_jpg.rf.58690ebe018b849e3d3f6ef8785ca148.jpg: 1024x1024 3 rickshaws, 1 motorbikes, 2 cars, 1 bicycles, 4 minibuss, Done. (0.020s)
image 3/450 /content/yolov5/data/images/Asraf_102_jpg.rf.40d695ed6f8db102530afcb11db466fa.jpg: 1024x1024 3 rickshaws, 1 motorbikes, 2 cars, 3 minibuss, Done. (0.020s)
image 4/450 /content/yolov5/data/images/Asra

We'll write a helper function to show the results:

In [ ]:
def load_image(img_path: Path, resize=True):
  img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (128, 256), interpolation = cv2.INTER_AREA)
  return img

def show_grid(image_paths):
  images = [load_image(img) for img in image_paths]
  images = torch.as_tensor(images)
  images = images.permute(0, 3, 1, 2)
  grid_img = torchvision.utils.make_grid(images, nrow=11)
  plt.figure(figsize=(24, 12))
  plt.imshow(grid_img.permute(1, 2, 0))
  plt.axis('off');

Here are some of the images along with the detected clothing:

In [ ]:
img_paths = list(Path("inference/output").glob("*.jpg"))[:22]
show_grid(img_paths)